In [18]:
import os

project_folder = './pytorch-peds'
os.makedirs(project_folder, exist_ok=True)

In [19]:
import shutil

shutil.copy('script.py', project_folder)

'./pytorch-peds/script.py'

In [1]:
#import sys
#sys.path.insert(0,'./')
# Install pycocotools
!git clone https://github.com/cocodataset/cocoapi.git
!ls
%cd cocoapi/PythonAPI
!ls
!pip install cython
!pip install numpy
!python setup.py build_ext install

fatal: destination path 'cocoapi' already exists and is not an empty directory.
PyTorch-object-detection-1.ipynb cocoapi
build                            config.json
/Users/gkv/Code/PT-Azure-maskrcnn/cocoapi/PythonAPI
Makefile             dist                 setup.py
PennFudanPed         engine.py            test.zip
__pycache__          pycocoDemo.ipynb     transforms.py
build                pycocoEvalDemo.ipynb utils.py
coco_eval.py         pycocotools          vision
coco_utils.py        pycocotools.egg-info
cocoapi              script.py
running build_ext
skipping 'pycocotools/_mask.c' Cython extension (up-to-date)
running install
running bdist_egg
running egg_info
writing pycocotools.egg-info/PKG-INFO
writing dependency_links to pycocotools.egg-info/dependency_links.txt
writing requirements to pycocotools.egg-info/requires.txt
writing top-level names to pycocotools.egg-info/top_level.txt
reading manifest file 'pycocotools.egg-info/SOURCES.txt'
writing manifest file 'pycocotools.e

In [2]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.85


In [4]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


In [5]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: gopalv-ws
Azure region: westus2
Subscription id: 15ae9cb6-95c1-483d-a0e3-b1a1a3b06324
Resource group: aifxdemo


In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target.
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-02-05T18:20:16.915000+00:00', 'errors': None, 'creationTime': '2020-02-05T18:19:06.976503+00:00', 'modifiedTime': '2020-02-05T18:20:28.120151+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 2, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'LowPriority', 'vmSize': 'STANDARD_NC6'}


In [7]:
import urllib.request
 
url = 'https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip'
 
urllib.request.urlretrieve(url, './test.zip')

('./test.zip', <http.client.HTTPMessage at 0x122cf2430>)

In [8]:
from zipfile import ZipFile

zip = ZipFile(file='./test.zip')
zip.extractall()

In [9]:
!ls PennFudanPed/

Annotation            PedMasks              readme.txt
PNGImages             added-object-list.txt


In [10]:
# get the default datastore
ds = ws.get_default_datastore()
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)

workspaceblobstore AzureBlob gopalvws3790775563 azureml-blobstore-e47496c6-9688-4277-a05b-ceb722514b9d


In [11]:
ds.upload('./PennFudanPed', target_path='data', overwrite=True)

Uploading an estimated of 512 files
Uploading ./PennFudanPed/PedMasks/FudanPed00072_mask.png
Uploading ./PennFudanPed/PedMasks/FudanPed00073_mask.png
Uploading ./PennFudanPed/PedMasks/PennPed00048_mask.png
Uploading ./PennFudanPed/PedMasks/PennPed00049_mask.png
Uploading ./PennFudanPed/added-object-list.txt
Uploading ./PennFudanPed/readme.txt
Uploading ./PennFudanPed/PedMasks/PennPed00001_mask.png
Uploading ./PennFudanPed/PedMasks/PennPed00093_mask.png
Uploading ./PennFudanPed/PedMasks/PennPed00092_mask.png
Uploading ./PennFudanPed/PedMasks/PennPed00076_mask.png
Uploading ./PennFudanPed/PedMasks/PennPed00077_mask.png
Uploading ./PennFudanPed/PedMasks/FudanPed00005_mask.png
Uploading ./PennFudanPed/PedMasks/FudanPed00004_mask.png
Uploading ./PennFudanPed/PedMasks/FudanPed00031_mask.png
Uploading ./PennFudanPed/PedMasks/FudanPed00030_mask.png
Uploading ./PennFudanPed/PedMasks/PennPed00042_mask.png
Uploading ./PennFudanPed/PedMasks/PennPed00043_mask.png
Uploading ./PennFudanPed/Annotation

Uploaded ./PennFudanPed/Annotation/FudanPed00072.txt, 69 files out of an estimated total of 512
Uploaded ./PennFudanPed/PedMasks/FudanPed00031_mask.png, 70 files out of an estimated total of 512
Uploaded ./PennFudanPed/Annotation/FudanPed00074.txt, 71 files out of an estimated total of 512
Uploading ./PennFudanPed/Annotation/PennPed00010.txt
Uploading ./PennFudanPed/Annotation/PennPed00011.txt
Uploading ./PennFudanPed/Annotation/PennPed00012.txt
Uploaded ./PennFudanPed/Annotation/FudanPed00012.txt, 72 files out of an estimated total of 512
Uploaded ./PennFudanPed/Annotation/PennPed00005.txt, 73 files out of an estimated total of 512
Uploading ./PennFudanPed/Annotation/PennPed00013.txt
Uploading ./PennFudanPed/Annotation/PennPed00014.txt
Uploaded ./PennFudanPed/Annotation/PennPed00001.txt, 74 files out of an estimated total of 512
Uploaded ./PennFudanPed/Annotation/PennPed00004.txt, 75 files out of an estimated total of 512
Uploaded ./PennFudanPed/PedMasks/PennPed00077_mask.png, 76 file

Uploaded ./PennFudanPed/PNGImages/FudanPed00017.png, 188 files out of an estimated total of 512
Uploading ./PennFudanPed/PNGImages/FudanPed00033.png
Uploaded ./PennFudanPed/PNGImages/FudanPed00005.png, 189 files out of an estimated total of 512
Uploading ./PennFudanPed/PNGImages/FudanPed00034.png
Uploaded ./PennFudanPed/PNGImages/FudanPed00028.png, 190 files out of an estimated total of 512
Uploading ./PennFudanPed/PNGImages/FudanPed00035.png
Uploaded ./PennFudanPed/PNGImages/FudanPed00018.png, 191 files out of an estimated total of 512
Uploading ./PennFudanPed/PNGImages/FudanPed00036.png
Uploaded ./PennFudanPed/PNGImages/FudanPed00025.png, 192 files out of an estimated total of 512
Uploading ./PennFudanPed/PNGImages/FudanPed00037.png
Uploaded ./PennFudanPed/PNGImages/FudanPed00004.png, 193 files out of an estimated total of 512
Uploading ./PennFudanPed/PNGImages/FudanPed00038.png
Uploaded ./PennFudanPed/PNGImages/FudanPed00034.png, 194 files out of an estimated total of 512
Uploading 

Uploaded ./PennFudanPed/PNGImages/FudanPed00042.png, 244 files out of an estimated total of 512
Uploading ./PennFudanPed/PNGImages/PennPed00015.png
Uploaded ./PennFudanPed/PNGImages/FudanPed00052.png, 245 files out of an estimated total of 512
Uploading ./PennFudanPed/PNGImages/PennPed00016.png
Uploaded ./PennFudanPed/PNGImages/FudanPed00058.png, 246 files out of an estimated total of 512
Uploading ./PennFudanPed/PNGImages/PennPed00017.png
Uploaded ./PennFudanPed/PNGImages/FudanPed00070.png, 247 files out of an estimated total of 512
Uploading ./PennFudanPed/PNGImages/PennPed00018.png
Uploaded ./PennFudanPed/PNGImages/FudanPed00049.png, 248 files out of an estimated total of 512
Uploading ./PennFudanPed/PNGImages/PennPed00019.png
Uploaded ./PennFudanPed/PNGImages/FudanPed00054.png, 249 files out of an estimated total of 512
Uploading ./PennFudanPed/PNGImages/PennPed00020.png
Uploaded ./PennFudanPed/PNGImages/FudanPed00057.png, 250 files out of an estimated total of 512
Uploading ./Penn

Uploaded ./PennFudanPed/PNGImages/PennPed00015.png, 301 files out of an estimated total of 512
Uploading ./PennFudanPed/PNGImages/PennPed00072.png
Uploaded ./PennFudanPed/PNGImages/PennPed00048.png, 302 files out of an estimated total of 512
Uploading ./PennFudanPed/PNGImages/PennPed00073.png
Uploaded ./PennFudanPed/PNGImages/PennPed00026.png, 303 files out of an estimated total of 512
Uploading ./PennFudanPed/PNGImages/PennPed00074.png
Uploaded ./PennFudanPed/PNGImages/PennPed00061.png, 304 files out of an estimated total of 512
Uploading ./PennFudanPed/PNGImages/PennPed00075.png
Uploaded ./PennFudanPed/PNGImages/PennPed00054.png, 305 files out of an estimated total of 512
Uploading ./PennFudanPed/PNGImages/PennPed00076.png
Uploaded ./PennFudanPed/PNGImages/PennPed00042.png, 306 files out of an estimated total of 512
Uploading ./PennFudanPed/PNGImages/PennPed00077.png
Uploaded ./PennFudanPed/PNGImages/PennPed00055.png, 307 files out of an estimated total of 512
Uploading ./PennFudanPe

Uploaded ./PennFudanPed/PedMasks/FudanPed00029_mask.png, 355 files out of an estimated total of 512
Uploading ./PennFudanPed/PedMasks/FudanPed00034_mask.png
Uploaded ./PennFudanPed/PedMasks/FudanPed00033_mask.png, 356 files out of an estimated total of 512
Uploading ./PennFudanPed/PedMasks/FudanPed00035_mask.png
Uploaded ./PennFudanPed/PedMasks/FudanPed00034_mask.png, 357 files out of an estimated total of 512
Uploading ./PennFudanPed/PedMasks/FudanPed00036_mask.png
Uploaded ./PennFudanPed/PedMasks/FudanPed00032_mask.png, 358 files out of an estimated total of 512
Uploading ./PennFudanPed/PedMasks/FudanPed00037_mask.png
Uploaded ./PennFudanPed/PedMasks/FudanPed00035_mask.png, 359 files out of an estimated total of 512
Uploading ./PennFudanPed/PedMasks/FudanPed00038_mask.png
Uploaded ./PennFudanPed/PedMasks/FudanPed00026_mask.png, 360 files out of an estimated total of 512
Uploading ./PennFudanPed/PedMasks/FudanPed00039_mask.png
Uploaded ./PennFudanPed/PedMasks/FudanPed00038_mask.png, 3

Uploaded ./PennFudanPed/PedMasks/PennPed00010_mask.png, 413 files out of an estimated total of 512
Uploading ./PennFudanPed/PedMasks/PennPed00021_mask.png
Uploaded ./PennFudanPed/PedMasks/FudanPed00064_mask.png, 414 files out of an estimated total of 512
Uploading ./PennFudanPed/PedMasks/PennPed00022_mask.png
Uploaded ./PennFudanPed/PedMasks/FudanPed00066_mask.png, 415 files out of an estimated total of 512
Uploaded ./PennFudanPed/PedMasks/PennPed00006_mask.png, 416 files out of an estimated total of 512
Uploading ./PennFudanPed/PedMasks/PennPed00023_mask.png
Uploading ./PennFudanPed/PedMasks/PennPed00024_mask.png
Uploaded ./PennFudanPed/PedMasks/PennPed00020_mask.png, 417 files out of an estimated total of 512
Uploaded ./PennFudanPed/PedMasks/PennPed00009_mask.png, 418 files out of an estimated total of 512
Uploading ./PennFudanPed/PedMasks/PennPed00025_mask.png
Uploading ./PennFudanPed/PedMasks/PennPed00026_mask.png
Uploaded ./PennFudanPed/PedMasks/PennPed00014_mask.png, 419 files ou

Uploaded ./PennFudanPed/PedMasks/PennPed00074_mask.png, 468 files out of an estimated total of 512
Uploading ./PennFudanPed/PedMasks/PennPed00082_mask.png
Uploaded ./PennFudanPed/PedMasks/PennPed00070_mask.png, 469 files out of an estimated total of 512
Uploading ./PennFudanPed/PedMasks/PennPed00083_mask.png
Uploaded ./PennFudanPed/PedMasks/PennPed00075_mask.png, 470 files out of an estimated total of 512
Uploading ./PennFudanPed/PedMasks/PennPed00084_mask.png
Uploaded ./PennFudanPed/PedMasks/PennPed00078_mask.png, 471 files out of an estimated total of 512
Uploading ./PennFudanPed/PedMasks/PennPed00085_mask.png
Uploaded ./PennFudanPed/PedMasks/PennPed00037_mask.png, 472 files out of an estimated total of 512
Uploading ./PennFudanPed/PedMasks/PennPed00086_mask.png
Uploaded ./PennFudanPed/PedMasks/PennPed00079_mask.png, 473 files out of an estimated total of 512
Uploading ./PennFudanPed/PedMasks/PennPed00087_mask.png
Uploaded ./PennFudanPed/PedMasks/PennPed00073_mask.png, 474 files out 

$AZUREML_DATAREFERENCE_89ce0dabcd5a441a90d6d7d1f00b9830

In [12]:
from azureml.core import Dataset
# create a FileDataset pointing to files in 'animals' folder and its subfolders recursively
datastore_paths = [(ds, 'data')]
penn_ds = Dataset.File.from_files(path=datastore_paths)
penn_ds.register(workspace=ws,
                 name='penn_ds',
                 description='Penn Fudan pedestrian data')

{
  "source": [
    "('workspaceblobstore', 'data')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "1cd2dd50-ae6d-44f9-81e2-f0044c862efe",
    "name": "penn_ds",
    "version": 1,
    "description": "Penn Fudan pedestrian data",
    "workspace": "Workspace.create(name='gopalv-ws', subscription_id='15ae9cb6-95c1-483d-a0e3-b1a1a3b06324', resource_group='aifxdemo')"
  }
}

In [13]:
dataset_name = 'penn_ds'

# Get a dataset by name
penn1_ds = Dataset.get_by_name(workspace=ws, name=dataset_name)

In [17]:
# from PIL import Image
# import os
# import glob


# with penn_ds.mount() as mount_context:
#     # list top level mounted files and folders in the dataset
#     imgs_path =(os.path.join(mount_context.mount_point, 'PNGImages'))
#     print(f'imgs_path is {imgs_path}')
#     print(list(sorted(os.listdir(imgs_path))))
#     Image.open(os.path.join(mount_context.mount_point, 'PNGImages/PennPed00036.png'))
penn1_ds.as_named_input('test').as_mount()

In [14]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image


class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transforms=None):
        self.dataset = dataset
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        with self.dataset.mount() as mount_context:
            self.imgs = list(sorted(os.listdir(os.path.join(mount_context.mount_point, "PNGImages"))))
            self.masks = list(sorted(os.listdir(os.path.join(mount_context.mount_point, "PedMasks"))))

    def __getitem__(self, idx):
        # load images ad masks
        with self.dataset.mount() as mount_context:
            img_path = os.path.join(mount_context.mount_point, "PNGImages", self.imgs[idx])
            mask_path = os.path.join(mount_context.mount_point, "PedMasks", self.masks[idx])
            img = Image.open(img_path).convert("RGB")
            # note that we haven't converted the mask to RGB,
            # because each color corresponds to a different instance
            # with 0 being background
            mask = Image.open(mask_path)

            mask = np.array(mask)
            # instances are encoded as different colors
            obj_ids = np.unique(mask)
            # first id is the background, so remove it
            obj_ids = obj_ids[1:]

            # split the color-encoded mask into a set
            # of binary masks
            masks = mask == obj_ids[:, None, None]

            # get bounding box coordinates for each mask
            num_objs = len(obj_ids)
            boxes = []
            for i in range(num_objs):
                pos = np.where(masks[i])
                xmin = np.min(pos[1])
                xmax = np.max(pos[1])
                ymin = np.min(pos[0])
                ymax = np.max(pos[0])
                boxes.append([xmin, ymin, xmax, ymax])

            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            # there is only one class
            labels = torch.ones((num_objs,), dtype=torch.int64)
            masks = torch.as_tensor(masks, dtype=torch.uint8)

            image_id = torch.tensor([idx])
            area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
            # suppose all instances are not crowd
            iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

            target = {}
            target["boxes"] = boxes
            target["labels"] = labels
            target["masks"] = masks
            target["image_id"] = image_id
            target["area"] = area
            target["iscrowd"] = iscrowd

            if self.transforms is not None:
                img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [15]:
PennFudanDataset(penn_ds)

In [16]:
!pip install --upgrade torchvision

Requirement already up-to-date: torchvision in /Users/gkv/miniconda3/envs/azureml/lib/python3.8/site-packages (0.5.0)


In [27]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

      
def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [18]:
!git clone https://github.com/pytorch/vision.git
%cd vision
!git checkout v0.3.0

!cp references/detection/utils.py ../
!cp references/detection/transforms.py ../
!cp references/detection/coco_eval.py ../
!cp references/detection/engine.py ../
!cp references/detection/coco_utils.py ../

Cloning into 'vision'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 6710 (delta 8), reused 15 (delta 0), pack-reused 6662
Receiving objects: 100% (6710/6710), 9.77 MiB | 9.85 MiB/s, done.
Resolving deltas: 100% (4487/4487), done.
/Users/gkv/Code/PT-Azure-maskrcnn/cocoapi/PythonAPI/vision
Note: checking out 'v0.3.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at be37608 version check against PyTorch's CUDA version


In [21]:
import shutil

files_to_copy = ['utils', 'transforms', 'coco_eval', 'engine', 'coco_utils']
for file in files_to_copy:
    shutil.copy('vision/references/detection/'+ file + '.py', project_folder)


In [22]:
from azureml.core import Experiment

experiment_name = 'pytorch-peds'
experiment = Experiment(ws, name=experiment_name)

In [26]:
from azureml.train.dnn import PyTorch

script_params = {
    '--num_epochs': 10,
    '--output_dir': './outputs'
}

estimator = PyTorch(source_directory=project_folder, 
                    script_params=script_params,
                    compute_target=compute_target,
                    entry_script='script.py',
                    use_gpu=True,
                    pip_packages=['torchvision>=0.5.0'])

WARNING - framework_version is not specified, defaulting to version 1.3.
WARNING - You have specified to install packages in your run. Note that you have overridden Azure ML's installation of the following packages: ['torchvision']. We cannot guarantee image build will succeed.


In [27]:
run = experiment.submit(estimator)
print(run)

Run(Experiment: pytorch-peds,
Id: pytorch-peds_1581106066_f89252bf,
Type: azureml.scriptrun,
Status: Starting)


In [ ]:
# to get more details of your run
print(run.get_details())

In [29]:
from azureml.widgets import RunDetails

RunDetails(run).show()
run.wait_for_completion(show_output=True)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

RunId: pytorch-peds_1581106066_f89252bf
Web View: https://ml.azure.com/experiments/pytorch-peds/runs/pytorch-peds_1581106066_f89252bf?wsid=/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/aifxdemo/workspaces/gopalv-ws

Streaming azureml-logs/20_image_build_log.txt

2020/02/07 20:08:01 Downloading source code...
2020/02/07 20:08:03 Finished downloading source code
2020/02/07 20:08:04 Creating Docker network: acb_default_network, driver: 'bridge'
2020/02/07 20:08:05 Successfully set up Docker network: acb_default_network
2020/02/07 20:08:05 Setting up Docker configuration...
2020/02/07 20:08:05 Successfully set up Docker configuration
2020/02/07 20:08:05 Logging in to registry: gopalvwsdebb3b12.azurecr.io
2020/02/07 20:08:06 Successfully logged into gopalvwsdebb3b12.azurecr.io
2020/02/07 20:08:06 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/02/07 20:08:06 Scanning for dependencies...
2020/02/07 20:08:07 Su

Executing transaction: ...working... done


  ERROR: Command errored out with exit status 1:
   command: /azureml-envs/azureml_0ed3bc1f720e34afdccfafeca630e0c1/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-ud8xfl9a/horovod/setup.py'"'"'; __file__='"'"'/tmp/pip-install-ud8xfl9a/horovod/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-jlb5sit4
       cwd: /tmp/pip-install-ud8xfl9a/horovod/
  Complete output (169 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-3.6
  creating build/lib.linux-x86_64-3.6/horovod
  copying horovod/__init__.py -> build/lib.linux-x86_64-3.6/horovod
  creating build/lib.linux-x86_64-3.6/horovod/keras
  copying horovod/keras/__init__.py -> build/lib.linux-x86_64-3.6/horovod/keras
  copying horovod/kera

  Created wheel for future: filename=future-0.17.1-py3-none-any.whl size=488732 sha256=90ea2437fc460b2fe809f5feb13c635fb53e06f53aad9255f9bb66a47e9cf9d0
  Stored in directory: /root/.cache/pip/wheels/c4/f0/e2/8e4ecc9e1b12a428a7657ba683576d3e79d0a75982f63e8fd2
  Created wheel for json-logging-py: filename=json_logging_py-0.2-py3-none-any.whl size=3923 sha256=072dfee6eec88c59666cd7cd847b2645912b1e37e20380bc95dacca3debde543
  Stored in directory: /root/.cache/pip/wheels/e2/1d/52/535a274b9c2ce7d4064838f2bdb62013801281ef7d7f21e2ee
  Created wheel for psutil: filename=psutil-5.6.7-cp36-cp36m-linux_x86_64.whl size=274805 sha256=2fe2b0dbc3b8071cfd7248e05635ed9187dba6f7a9bd1480f592a54ed48194e4
  Stored in directory: /root/.cache/pip/wheels/b2/b3/a3/551db06cebc9141c437c6b58aa751d3670471494e9f6d1428d
  Created wheel for pyyaml: filename=PyYAML-5.3-cp36-cp36m-linux_x86_64.whl size=44229 sha256=12d7c0cbdc6219e9c44542e282e139088586d97c75d631f19e063202a148fb95
  Stored in directory: /root/.cache/pip/w

7fc58980ccdc: Pushed
09f5cc6238d6: Pushed
76ce30d10f04: Pushed
ae1aecc127c3: Pushed
0e6f845ffcee: Pushed
c63d8cf11fb9: Pushed
0cf1d4dd1413: Pushed
a31485cd6e70: Pushed
808fd332a58a: Pushed
b16af11cbf29: Pushed
37b9a4b22186: Pushed
e0b3afb09dc3: Pushed
6c01b5a53aac: Pushed
2c6ac8e5063e: Pushed
078e76f747e1: Pushed
1db09913a256: Pushed
cc967c529ced: Pushed
e6f174f76be4: Pushed
2e282f599fd6: Pushed
latest: digest: sha256:2e41c42cb1c0149703022026098f88e996707f36ae2036445921fd4f79933849 size: 5151
2020/02/07 20:21:20 Successfully pushed image: gopalvwsdebb3b12.azurecr.io/azureml/azureml_3f87844078b25a93e23b78fecf0aedc9:latest
2020/02/07 20:21:21 Step ID: acb_step_0 marked as successful (elapsed time in seconds: 448.183365)
2020/02/07 20:21:21 Populating digests for step ID: acb_step_0...
2020/02/07 20:21:28 Successfully populated digests for step ID: acb_step_0
2020/02/07 20:21:28 Step ID: acb_step_1 marked as successful (elapsed time in seconds: 345.916678)
2020/02/07 20:21:28 The followin

...


Streaming azureml-logs/75_job_post-tvmps_8f29b192b50fc41a6194d0f597d41e6b488308096a4f839b83e2bd305534065c_p.txt



WARNING - Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', OSError("(60, 'ETIMEDOUT')"))': /azureml/ExperimentRun/dcid.pytorch-peds_1581106066_f89252bf/azureml-logs/75_job_post-tvmps_8f29b192b50fc41a6194d0f597d41e6b488308096a4f839b83e2bd305534065c_p.txt?sv=2019-02-02&sr=b&sig=YPOm%2FN5Sa421zbkSAJUorjaDux9%2FV2KYviuXrwlBYJY%3D&st=2020-02-07T20%3A40%3A35Z&se=2020-02-08T04%3A50%3A35Z&sp=r


bash: /azureml-envs/azureml_0ed3bc1f720e34afdccfafeca630e0c1/lib/libtinfo.so.5: no version information available (required by bash)
Starting job release. Current time:2020-02-07T20:38:52.903617
Logging experiment finalizing status in history service.
Starting the daemon thread to refresh tokens in background for process with pid = 165
Job release is complete. Current time:2020-02-07T20:38:55.793905


WARNING - Retrying (Retry(total=2, connect=2, read=3, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x122fa3cd0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known')': /history/v1.0/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/aifxdemo/providers/Microsoft.MachineLearningServices/workspaces/gopalv-ws/experiments/pytorch-peds/runs/pytorch-peds_1581106066_f89252bf/details
WARNING - Retrying (Retry(total=2, connect=2, read=3, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x122f3e4c0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known')': /history/v1.0/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/aifxdemo/providers/Microsoft.MachineLearningServices/workspaces/gopalv-ws/experiments/pytorch-peds/runs

AttributeError: 'NoneType' object has no attribute 'details'

In [19]:
import sys
sys.path.insert(0,'./')

In [20]:
from platform import python_version

print(python_version())
help("modules")

3.8.1

Please wait a moment while I gather a list of all available modules...



/Users/gkv/miniconda3/envs/azureml/lib/python3.8/site-packages/IPython/kernel/__init__.py:12: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  warn("The `IPython.kernel` package has been deprecated since IPython 4.0."
/Users/gkv/miniconda3/envs/azureml/lib/python3.8/pkgutil.py:107: VisibleDeprecationWarning: zmq.eventloop.minitornado is deprecated in pyzmq 14.0 and will be removed.
    Install tornado itself to use zmq with the tornado IOLoop.
    
  yield from walk_packages(path, info.name+'.', onerror)


Cython              asynchat            inspect             random
IPython             asyncio             io                  re
OpenSSL             asyncore            ipaddress           readline
PIL                 atexit              ipykernel           reprlib
PyQt5               attr                ipykernel_launcher  requests
__future__          audioop             ipython_genutils    requests_oauthlib
_abc                automl              ipywidgets          resource
_ast                autoreload          isodate             rlcompleter
_asyncio            azureml             itertools           rmagic
_bisect             backcall            jaraco              runpy
_blake2             backports           jedi                sched
_bootlocale         base64              jeepney             secrets
_bz2                bdb                 jinja2              secretstorage
_cffi_backend       binascii            jmespath            select
_codecs             binhex           

/Users/gkv/miniconda3/envs/azureml/lib/python3.8/pkgutil.py:92: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid module was deprecated in Matplotlib 2.1 and will be removed two minor releases later. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist, which provide the same functionality instead.
  __import__(info.name)


In [28]:
# from engine import train_one_epoch, evaluate
# import utils
# import transforms as T

# def get_transform(train):
#     transforms = []
#     # converts the image, a PIL image, into a PyTorch Tensor
#     transforms.append(T.ToTensor())
#     if train:
#         # during training, randomly flip the training images
#         # and ground-truth for data augmentation
#         transforms.append(T.RandomHorizontalFlip(0.5))
#     return T.Compose(transforms)


In [33]:
# # use our dataset and defined transformations
# dataset = PennFudanDataset(penn_ds, get_transform(train=True))
# dataset_test = PennFudanDataset(penn_ds, get_transform(train=False))

# # split the dataset in train and test set
# torch.manual_seed(1)
# indices = torch.randperm(len(dataset)).tolist()
# dataset = torch.utils.data.Subset(dataset, indices[:-50])
# dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# # define training and validation data loaders
# data_loader = torch.utils.data.DataLoader(
#     dataset, batch_size=2, shuffle=True, num_workers=4,
#     collate_fn=utils.collate_fn)

# data_loader_test = torch.utils.data.DataLoader(
#     dataset_test, batch_size=1, shuffle=False, num_workers=4,
#     collate_fn=utils.collate_fn)

In [34]:
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# # our dataset has two classes only - background and person
# num_classes = 2

# # get the model using our helper function
# model = get_instance_segmentation_model(num_classes)
# # move model to the right device
# model.to(device)

# # construct an optimizer
# params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.SGD(params, lr=0.005,
#                             momentum=0.9, weight_decay=0.0005)

# # and a learning rate scheduler which decreases the learning rate by
# # 10x every 3 epochs
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
#                                                step_size=3,
#                                                gamma=0.1)

In [1]:
# num_epochs = 10

# for epoch in range(num_epochs):
#     # train for one epoch, printing every 10 iterations
#     train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
#     # update the learning rate
#     lr_scheduler.step()
#     # evaluate on the test dataset
#     evaluate(model, data_loader_test, device=device)

NameError: name 'train_one_epoch' is not defined

In [ ]:
# # pick one image from the test set
# img, _ = dataset_test[0]
# # put the model in evaluation mode
# model.eval()
# with torch.no_grad():
#     prediction = model([img.to(device)])

In [ ]:
# Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())